In [1]:
# importamos las librerías necesarias y establecemos conexión
import sim  # librería para conectar con CoppeliaSim
import numpy as np
from time import sleep
def connect(port):
# Establece la conexión a VREP
# port debe coincidir con el puerto de conexión en VREP
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID
# Controladores
# Requerimos los handlers para los joints, la ventosa y el sensor de succión (Permite saber si el objeto está cerca)
# Requerimos los handlers para los joints, la ventosa y el sensor de succión (Permite saber si el objeto está cerca)
clientID = connect(19999)

conectado a 19999


In [2]:
class Drone():
    def __init__(self,ClientID,Area):
        self.ClientID = ClientID
        self.Size = Area

    def Handles(self, Dummy): 	#Funcion para obtener los handles
        self.ThermalCamera = sim.simxGetObjectHandle(self.clientID, Dummy, sim.simx_opmode_blocking)[-1]

        #self.__InitalizeSensors()

    def __InitalizeSensors(self):
        '''Funcion para inicializar los sensores de vision
        se necesitan como minimo 0.02 segundos entre el modo streaming y buffer'''

        #Sensores para detectar el bloque
        sim.simxGetVisionSensorImage(self.clientID,self.ThermalCamera,0,sim.simx_opmode_streaming)
        sleep(0.02)
        _,Resolution,_ = sim.simxGetVisionSensorImage(self.clientID,self.ThermalCamera,0,sim.simx_opmode_buffer)

    def GetThermalCamera(self,Resolution):
        _, _, RGB = sim.simxGetVisionSensorImage(self.clientID,self.ThermalCamera,0,sim.simx_opmode_buffer) 
        img = np.array(RGB,dtype=np.uint8)
        img.resize([Resolution[0],Resolution[1],3])
        return img

In [3]:
searchArea = (1, 1)
drone = Drone(clientID, searchArea)
Start = sim.simxGetObjectHandle(clientID, 'Dummy', sim.simx_opmode_blocking)[-1]
Target = sim.simxGetObjectHandle(clientID, 'Quadcopter_target', sim.simx_opmode_blocking)[-1]
Body = sim.simxGetObjectHandle(clientID, 'Quadcopter', sim.simx_opmode_blocking)[-1]

In [4]:
sim.simxGetObjectPosition(clientID, Target, Start, sim.simx_opmode_streaming)[-1]
TargetPosition = sim.simxGetObjectPosition(clientID, Target, Start, sim.simx_opmode_buffer)[-1]

sim.simxGetObjectOrientation(clientID, Target, Start, sim.simx_opmode_streaming)[-1]

sim.simxGetObjectPosition(clientID, Body, Start, sim.simx_opmode_streaming)
sleep(0.2)

In [5]:
def ValidatePosition():
    while True:
        BodyPosition = sim.simxGetObjectPosition(clientID, Body, Start, sim.simx_opmode_buffer)[-1]
        BodyPosition = [round(x, 2) for x in BodyPosition]

        TargetPosition = sim.simxGetObjectPosition(clientID, Target, Start, sim.simx_opmode_buffer)[-1]
        TargetPosition = [round(x, 2) for x in TargetPosition]

        if BodyPosition == TargetPosition:
            break


def ValidateOrientation():
    while True:
        BodyOrientation = sim.simxGetObjectOrientation(clientID, Body, Start, sim.simx_opmode_buffer)[-1]
        BodyOrientation = [round(x, 2) for x in BodyOrientation]

        TargetOrientation = sim.simxGetObjectOrientation(clientID, Target, Start, sim.simx_opmode_buffer)[-1]
        TargetOrientation = [round(x, 2) for x in TargetOrientation]

        if BodyOrientation == TargetOrientation:
            break

In [6]:

Altura = 2.4

Z = list(np.arange(0, Altura, 0.2))

for Depth in Z:
    sleep(0.05)
    sim.simxSetObjectPosition(clientID, Target, Start, [0, 0, Depth], sim.simx_opmode_oneshot)

X = list(np.arange(0, 0.1, 0.005))
Y = list(np.arange(0, 5, 0.05))
referencia = sim.simxGetObjectOrientation(clientID, Target, Start, sim.simx_opmode_buffer)[-1]
Ubicacion = sim.simxGetObjectOrientation(clientID, Target, Start, sim.simx_opmode_buffer)[-1]
RotationAngle = list(np.arange(referencia[-1], 0, 0.0001))
RotationAngle1 = list(np.arange(-1.570796012878418, 0, 0.0001))

sleep(2)

i, j = 0, 0
SleepTime = 0.05
SleepTime1 = 0.03

Detected = False

In [ ]:
while True:
    # Izq a Derecha
    for j in Y:
        sleep(SleepTime)
        sim.simxSetObjectPosition(clientID, Target, Start, [i, j, Altura], sim.simx_opmode_oneshot)
        
    # ValidatePosition()


    sleep(1)

    # Rotacion 45 grados a izq
    for Angle in RotationAngle:
        sim.simxSetObjectOrientation(clientID, Target, Start, [0, 0, Angle], sim.simx_opmode_oneshot)

    sleep(1)

    # Mov eje x
    for Temp in X:
        i -= Temp
        sleep(SleepTime1)
        sim.simxSetObjectPosition(clientID, Target, Start, [i, j, Altura], sim.simx_opmode_oneshot)
    # ValidatePosition()

    sleep(1)

    # Rotacion 45 grados a izq
    for Angle in reversed(RotationAngle1):
        sim.simxSetObjectOrientation(clientID, Target, Start, [0, 0, -Angle], sim.simx_opmode_oneshot)

    sleep(1)

    # Mov eje -Y
    for j in reversed(Y):
        sleep(SleepTime)
        sim.simxSetObjectPosition(clientID, Target, Start, [i, j, Altura], sim.simx_opmode_oneshot)
        
    # ValidatePosition()

    sleep(1)

    for Angle in RotationAngle1:
        sim.simxSetObjectOrientation(clientID, Target, Start, [0, 0, -Angle], sim.simx_opmode_oneshot)

    for Temp in X:
        i -= Temp
        sleep(SleepTime1)
        sim.simxSetObjectPosition(clientID, Target, Start, [i, j, Altura], sim.simx_opmode_oneshot)
    # ValidatePosition()

    sleep(1)

    # Rotacion 45 grados a izq
    for Angle in reversed(RotationAngle):
        sim.simxSetObjectOrientation(clientID, Target, Start, [0, 0, Angle], sim.simx_opmode_oneshot)